# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [25]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [26]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [27]:
# frequency table for prestige and whether or not someone was admitted
print df.prestige.value_counts()
print df.admit.value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64
0    271
1    126
Name: admit, dtype: int64


## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [28]:
dummy_ranks = pd.get_dummies(df.prestige.astype(int),prefix = "prestige")
print dummy_ranks.head()

   prestige_1  prestige_2  prestige_3  prestige_4
0           0           0           1           0
1           0           0           1           0
2           1           0           0           0
3           0           0           0           1
4           0           0           0           1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: We need k-1, or 3 

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [29]:
admitted = float(df.admit[df.admit == 1].sum()) 
print admitted,"people were admitted"

not_admitted = float(df.admit[df.admit == 0].count())
print not_admitted,"people were NOT admitted"

odds_admitted = admitted/not_admitted 
print "Odds admitted are",round(odds_admitted,2),"to 1"

odds_not_admitted = not_admitted/admitted
print "Odds not admitted are",round(odds_not_admitted,2),"to 1"

odds_ratio = odds_admitted/odds_not_admitted
print "Odds ratio is",odds_ratio

126.0 people were admitted
271.0 people were NOT admitted
Odds admitted are 0.46 to 1
Odds not admitted are 2.15 to 1
Odds ratio is 0.216173527049


In [30]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


In [31]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc["prestige_1"], handCalc.admit)

admit,0,1
prestige_1,,
0,243,93
1,28,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [32]:
odds_admit_prestige_1 = 33./28
print """
Your odds of getting into grad school are %f to 1 
if you attended a top-ranked undergrad school""" % odds_admit_prestige_1



Your odds of getting into grad school are 1.178571 to 1 
if you attended a top-ranked undergrad school


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [33]:
odds_admit_notprestige_1 = 93./243
print """
Your odds of getting into grad school are %f to 1 
if you did NOT attend a top-ranked undergrad school""" % odds_admit_notprestige_1


Your odds of getting into grad school are 0.382716 to 1 
if you did NOT attend a top-ranked undergrad school


#### 3.3 Calculate the odds ratio

In [34]:
odds_ratio = odds_admit_prestige_1 / odds_admit_notprestige_1
print """
The odds ratio for being admitted to a grad school based on 
whether or not the undergrad school was top-ranked is %f.""" % odds_ratio


The odds ratio for being admitted to a grad school based on 
whether or not the undergrad school was top-ranked is 3.079493.


#### 3.4 Write this finding in a sentenance: 

Answer: Those who graduate from a top-ranked undergrad program are 3x more likely to be admitted into a grad school than those who did not.

#### 3.5 Print the cross tab for prestige_4

In [35]:
pd.crosstab(handCalc.prestige_4, handCalc.admit)

admit,0,1
prestige_4,,
0,216,114
1,55,12


#### 3.6 Calculate the OR 

In [36]:
odds_admit_prestige_4 = 12./55
print """
Your odds of getting into grad school are %f to 1 
if you attended a bottom-ranked undergrad school""" % odds_admit_prestige_4

odds_admit_notprestige_4 = 114./216
print """
Your odds of getting into grad school are %f to 1 
if you did NOT attend a bottom-ranked undergrad school""" % odds_admit_notprestige_4

odds_ratio = odds_admit_prestige_4 / odds_admit_notprestige_4
print """
The odds ratio for being admitted to a grad school based on 
whether or not the undergrad school was bottom-ranked is %f.""" % odds_ratio


Your odds of getting into grad school are 0.218182 to 1 
if you attended a bottom-ranked undergrad school

Your odds of getting into grad school are 0.527778 to 1 
if you did NOT attend a bottom-ranked undergrad school

The odds ratio for being admitted to a grad school based on 
whether or not the undergrad school was bottom-ranked is 0.413397.


#### 3.7 Write this finding in a sentence

Answer: Those who graduate from a bottom-ranked undergrad program are 59% less  likely to be admitted into a grad school than those who did not.

## Part 4. Analysis

In [37]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1':])
print data.head()

   admit    gre   gpa  prestige_1  prestige_2  prestige_3  prestige_4
0      0  380.0  3.61           0           0           1           0
1      1  660.0  3.67           0           0           1           0
2      1  800.0  4.00           1           0           0           0
3      1  640.0  3.19           0           0           0           1
4      0  520.0  2.93           0           0           0           1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [38]:
# manually add the intercept
data['intercept'] = 1.0


#### 4.1 Set the covariates to a variable called train_cols

In [39]:
train_cols = data.columns[1:]
print train_cols

Index([u'gre', u'gpa', u'prestige_1', u'prestige_2', u'prestige_3',
       u'prestige_4', u'intercept'],
      dtype='object')


#### 4.2 Fit the model

In [40]:
lm = sm.Logit(data['admit'], data[train_cols])
result = lm.fit()


Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 7


#### 4.3 Print the summary results

In [41]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sat, 17 Jun 2017   Pseudo R-squ.:                 0.08166
Time:                        15:12:15   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
gre            0.0022      0.001      2.028      0.043    7.44e-05       0.004
gpa            0.7793      0.333      2.344      0.019       0.128       1.431
prestige_1    -0.0609   7.99e+06  -7.63e-09      1.000   -1.57e+07    1.57e+07
prestige_2    -0.7411   7.99e+06  -9.28e-08      1.000   -1.57e+07    1.57e+07
prestige_3    -1.3996   7.99e+06  -1.75e-07      1.000   -1.57e+07    1.57e+07
prestige_4    -1.6143   7.99e+06  -2.02e-07      1.000   -1.57e+07    1.57e+07
intercept     -3.8159   7.99e+06  -4.78e-07      1.000   -1.57e+07    1.57e+07
==============================================================================
"""

#### 4.4 Calculate the odds ratios of the coefficients and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [50]:
coefficients = result.params

coef_conf = result.conf_int()

OR = np.exp(coefficients)

OR_conf = np.exp(coef_conf)

print OR
print OR_conf

gre           1.002221
gpa           2.180027
prestige_1    0.940893
prestige_2    0.476607
prestige_3    0.246695
prestige_4    0.199023
intercept     0.022017
dtype: float64
                   0         1
gre         1.000074  1.004372
gpa         1.136120  4.183113
prestige_1  0.000000       inf
prestige_2  0.000000       inf
prestige_3  0.000000       inf
prestige_4  0.000000       inf
intercept   0.000000       inf


C:\Users\anne1\Anaconda2\lib\site-packages\ipykernel\__main__.py:7: RuntimeWarning: overflow encountered in exp


In [46]:
conf = result.conf_int()
conf.head(6)

,0,1
gre,7.444681e-05,4.362360e-03
gpa,1.276188e-01,1.431056e+00
prestige_1,-1.565407e+07,1.565407e+07
prestige_2,-1.565407e+07,1.565407e+07
prestige_3,-1.565407e+07,1.565407e+07
prestige_4,-1.565407e+07,1.565407e+07


In [48]:
conf['OR'] = coefficients
conf.columns = ['2.5%','97.5', 'OR']
print conf


                    2.5%          97.5        OR
gre         7.444681e-05  4.362360e-03  0.002218
gpa         1.276188e-01  1.431056e+00  0.779337
prestige_1 -1.565407e+07  1.565407e+07 -0.060926
prestige_2 -1.565407e+07  1.565407e+07 -0.741063
prestige_3 -1.565407e+07  1.565407e+07 -1.399603
prestige_4 -1.565407e+07  1.565407e+07 -1.614337
intercept  -1.565407e+07  1.565406e+07 -3.815928


#### 4.5 Interpret the OR of Prestige_2

Answer: Those who attended a prestige 2 school are 50% less likely to get admitted than those with prestige 1 schools. 


#### 4.6 Interpret the OR of GPA

Answer: Each one unit increase in GPA increases the chances of getting into grad school by 2.18 units of probability 

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [ ]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [ ]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]),columns = ['gre','gpas','prestigev2','intercept'])
combos.head()

#### 5.1 Recreate the dummy variables

In [ ]:
combos['prestigev2'].astype(int, copy = True)

In [ ]:
combos.head()

In [ ]:
# recreate the dummy variables

dummy_ranks_2 = pd.get_dummies(combos['prestigev2'].astype(int), prefix = 'prestige')

print dummy_ranks_2.head()

# keep only what we need for making predictions

new_cols_to_keep = ['gre', 'gpas','intercept']
new_data = combos[new_cols_to_keep].join(dummy_ranks_2.loc[:, 'prestige_1':])

print new_data.head()



# create a clean data frame for the regression
#cols_to_keep = ['admit', 'gre', 'gpa']
#data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
#print data.head()

#### 5.2 Make predictions on the enumerated dataset

In [ ]:
result.predict

new_data




#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.